## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import csv

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Hilo,19.7297,-155.0900,299.15,47,40,4.12,US,scattered clouds
1,1,Haines Junction,60.7522,-137.5108,247.15,70,73,0.51,CA,broken clouds
2,2,Hay River,60.8156,-115.7999,238.15,61,10,2.06,CA,clear sky
3,3,Souillac,-20.5167,57.5167,298.15,83,40,2.06,MU,scattered clouds
4,4,Coos Bay,43.3665,-124.2179,283.15,81,90,2.06,US,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Hilo,19.7297,-155.0900,299.15,47,40,4.12,US,scattered clouds
1,1,Haines Junction,60.7522,-137.5108,247.15,70,73,0.51,CA,broken clouds
2,2,Hay River,60.8156,-115.7999,238.15,61,10,2.06,CA,clear sky
3,3,Souillac,-20.5167,57.5167,298.15,83,40,2.06,MU,scattered clouds
4,4,Coos Bay,43.3665,-124.2179,283.15,81,90,2.06,US,overcast clouds
5,5,Hermanus,-34.4187,19.2345,292.59,91,100,1.57,ZA,light rain
6,6,Saint-Philippe,-21.3585,55.7679,299.15,73,75,3.09,RE,broken clouds
7,7,Masty,53.4122,24.5387,251.95,92,0,0.90,BY,clear sky
9,9,Inhambane,-23.8650,35.3833,297.98,88,100,3.05,MZ,overcast clouds
10,10,Aykhal,66.0000,111.5000,237.76,82,3,4.11,RU,clear sky


In [5]:
# 4a. Determine if there are any empty rows.

preferred_cities_df.count()

City_ID                631
City                   631
Lat                    631
Lng                    631
Max Temp               631
Humidity               631
Cloudiness             631
Wind Speed             631
Country                628
Current Description    631
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df= preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hilo,US,299.15,scattered clouds,19.7297,-155.0900,
1,Haines Junction,CA,247.15,broken clouds,60.7522,-137.5108,
2,Hay River,CA,238.15,clear sky,60.8156,-115.7999,
3,Souillac,MU,298.15,scattered clouds,-20.5167,57.5167,
4,Coos Bay,US,283.15,overcast clouds,43.3665,-124.2179,
5,Hermanus,ZA,292.59,light rain,-34.4187,19.2345,
6,Saint-Philippe,RE,299.15,broken clouds,-21.3585,55.7679,
7,Masty,BY,251.95,clear sky,53.4122,24.5387,
9,Inhambane,MZ,297.98,overcast clouds,-23.8650,35.3833,
10,Aykhal,RU,237.76,clear sky,66.0000,111.5000,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna()


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
nfo_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig
